In [1]:
import keras.backend.tensorflow_backend as K
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from common_helper import *
from nltk_helper import *
from keras_helper import *

/root/anaconda3/envs/ml_python/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_dr = "../input/train.json"
test_dr = "../input/test.json"
traindf, testdf = data_load(train_dr, test_dr)

In [3]:
instances = []
for ingredients in traindf['ingredients']:
    instance = []
    for ingredient in ingredients:
        word = []
        for pos in ingredient.split(" "):
            instance.append(WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', pos)))
    instances.append(''.join(instance))

In [4]:
traindf['ingredients_string_char'] = instances
x_train_char = traindf['ingredients_string_char'].tolist()

In [5]:
instances = []
for ingredients in testdf['ingredients']:
    instance = []
    for ingredient in ingredients:
        word = []
        for pos in ingredient.split(" "):
            instance.append(WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', pos)))
    instances.append(''.join(instance))

In [6]:
testdf['ingredients_string_char'] = instances
x_test_char = testdf['ingredients_string_char'].tolist()

In [7]:
instances = []
for ingredients in traindf['ingredients']:
    instance = []
    for ingredient in ingredients:
        word = []
        for pos in ingredient.split(" "):
            instance.append(WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', pos)))
    instances.append(instance)

In [8]:
traindf['ingredients_string_split'] = instances
traindf['ingredients_string_split'] = [','.join(i).strip().lower() for i in traindf['ingredients_string_split']] 
x_train_word = traindf['ingredients_string_split'].tolist()

In [9]:
instances = []
for ingredients in testdf['ingredients']:
    instance = []
    for ingredient in ingredients:
        word = []
        for pos in ingredient.split(" "):
            instance.append(WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', pos)))
    instances.append(instance)

In [10]:
testdf['ingredients_string_split'] = instances
testdf['ingredients_string_split'] = [','.join(i).strip().lower() for i in testdf['ingredients_string_split']]
x_test_word = testdf['ingredients_string_split'].tolist()

In [11]:
x_train, y_train = word_data_preprocessing(traindf)
x_test, _ = word_data_preprocessing(testdf)

In [12]:
#tfidf = TfidfVectorizer(analyzer=lambda d: d.split(','), ngram_range=(1,2)).fit(x_train)
tfidf = TfidfVectorizer(analyzer=lambda d: d.split(',')).fit(x_train)

x_train = tfidf.fit_transform(x_train).toarray()
x_test = tfidf.transform(x_test).toarray()

lb = preprocessing.LabelBinarizer()
lb.fit(traindf.cuisine.values)
y_train = lb.transform(traindf.cuisine.values)

In [13]:
tfidf = TfidfVectorizer(analyzer=lambda d: d.split(',')).fit(x_train_word)

x_train_word = tfidf.fit_transform(x_train_word).toarray()
x_test_word = tfidf.transform(x_test_word).toarray()

In [14]:
countvec = CountVectorizer(analyzer='char').fit(x_train_char)

x_train_char = countvec.fit_transform(x_train_char).toarray()
x_test_char = countvec.transform(x_test_char).toarray()

In [15]:
x_train = np.concatenate((x_train, x_train_word), axis=1)
x_test = np.concatenate((x_test, x_test_word), axis=1)

In [16]:
x_train = np.concatenate((x_train, x_train_char), axis=1)
x_test = np.concatenate((x_test, x_test_char), axis=1)

In [17]:
x_train.shape

(39774, 9712)

In [ ]:
a = []
for i in range(5):
    with K.tf.device('/gpu:0'):
        mdl = load_model(x_train)
        mdl.fit(x_train, y_train, nb_epoch=60, batch_size=4000,validation_split=0.3)
        train_predict = mdl.predict(x_train)
        y_test = mdl.predict(x_test)     
        a.append(y_test)    
        predictions = lb.inverse_transform(y_test)

/root/strawberry/code/keras_helper.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(540, activation="relu", input_shape=(9712,), kernel_initializer="glorot_uniform")`
  model.add(Dense(540, init='glorot_uniform', activation='relu', input_shape=(x_train.shape[1],)))
/root/strawberry/code/keras_helper.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(180, activation="relu", kernel_initializer="glorot_uniform")`
  model.add(Dense(180, init='glorot_uniform', activation='relu'))
/root/anaconda3/envs/ml_python/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 27841 samples, validate on 11933 samples
Epoch 1/60
27841/27841 [==============================] - 4s 146us/step - loss: 2.8372 - acc: 0.1740 - val_loss: 2.5247 - val_acc: 0.3329
Epoch 2/60
27841/27841 [==============================] - 3s 111us/step - loss: 2.5120 - acc: 0.2849 - val_loss: 2.2519 - val_acc: 0.4259
Epoch 3/60
27841/27841 [==============================] - 3s 113us/step - loss: 2.2528 - acc: 0.3865 - val_loss: 1.9881 - val_acc: 0.4701
Epoch 4/60
27841/27841 [==============================] - 3s 121us/step - loss: 2.0291 - acc: 0.4490 - val_loss: 1.7964 - val_acc: 0.5067
Epoch 5/60
27841/27841 [==============================] - 3s 113us/step - loss: 1.8431 - acc: 0.4946 - val_loss: 1.6204 - val_acc: 0.5494
Epoch 6/60
27841/27841 [==============================] - 3s 110us/step - loss: 1.6838 - acc: 0.5330 - val_loss: 1.4652 - val_acc: 0.5896
Epoch 7/60
27841/27841 [==============================] - 3s 122us/step - loss: 1.5328 - acc: 0.5669 - val_loss: 1.3281 - 

In [25]:
result = lb.inverse_transform(sum(a))
save_submission(testdf, predictions)

저장했다.
